# Módulo de CoquiTTS Remoto. 🗣
Criado para suprir o a sinteze de voz no mod Minecraft Narrator - Stanley Parable. Feito para Felps.

Esse modulo deve ser utilizado em conjunto com o beckend do mod, adaptado por Keyditor, para que seja eficaz!

# Configurações Inportantes!


1.   Altere o tipo de runtime do Google Colab para **T4 GPU**. Para que o módulo possa utilizar o processamento CUDA e acelerar a produção do audio de narração.
2.   Dentro do minecraft, na tela de configuração do mod, deve-se inserir o **URL NGROK** no campo de **Elevenlabs API KEY**. O **URL NGROK** é gerado ao acionar o módulo e exposto no console.

3.   O campo **Voice** deve ser preenchido com o nome do arquivo de voz (sem .wav ex.: *Brian3* ), que por sua vez precisam estar dentro de **[../ src / ttsoff /]** na pasta do **backend**.



---



# Essencial:

Execute o codigo a seguir para Instalar as dependências

In [ ]:
!pip install flask flask-ngrok
!pip install tts
!pip install sounddevice
!pip install pyngrok
!ngrok authtoken "2roc2fjp5i4TP8rimezsAYAH0QP_7Fud8Zb5UQXZUcbSUXvFW"


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 50.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 96.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 21.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 14.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 109.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 938.0/938.0 kB 33.7 

# Módulo

In [ ]:
from flask import Flask, request, send_file, jsonify
from pyngrok import ngrok
import os
from werkzeug.utils import secure_filename
from TTS.api import TTS
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = '/content/drive/My Drive/speaker_wavs'  # Pasta no Google Drive

# Garante que a pasta para os uploads existe
os.makedirs(app.config['UPLOAD_FOLDER'], exist_ok=True)

tts = TTS(model_name="tts_models/multilingual/multi-dataset/xtts_v2", progress_bar=True).to(device)
#tts.model = tts.model.to(device)

@app.route('/synthesize', methods=['POST'])
def synthesize():
    try:
        # Obter texto e validar entrada
        text = request.form.get('text')
        if not text:
            return jsonify({"error": "Texto é obrigatório."}), 400

        # Salvar o arquivo de voz enviado (speaker_wav)
        speaker_wav = request.files.get('speaker_wav')
        if not speaker_wav:
            return jsonify({"error": "O arquivo speaker_wav é obrigatório."}), 400

        speaker_wav_filename = secure_filename(speaker_wav.filename)
        speaker_wav_path = os.path.join(app.config['UPLOAD_FOLDER'], speaker_wav_filename)
        speaker_wav.save(speaker_wav_path)

        # Gerar o áudio usando o arquivo enviado
        output_file = '/content/drive/My Drive/output.wav'
        tts.tts_to_file(text=text, file_path=output_file, speaker_wav=speaker_wav_path, language="pt")

        return send_file(output_file, as_attachment=True)

    except Exception as e:
        return jsonify({"error": str(e)}), 500

# Inicia o servidor Flask com ngrok
if __name__ == '__main__':
    # Cria o túnel ngrok
    public_url = ngrok.connect(5000).public_url
    print(f"Servidor público do ngrok rodando em: {public_url}")

    # Inicia o servidor Flask
    app.run(host='0.0.0.0', port=5000)

Usando dispositivo: cuda
 > You must confirm the following:
 | > "I have purchased a commercial license from Coqui: licensing@coqui.ai"
 | > "Otherwise, I agree to the terms of the non-commercial CPML: https://coqui.ai/cpml" - [y/n]
 | | > y
 > Downloading model to /root/.local/share/tts/tts_models--multilingual--multi-dataset--xtts_v2


100%|█████████▉| 1.87G/1.87G [00:44<00:00, 41.9MiB/s]
100%|██████████| 1.87G/1.87G [00:44<00:00, 41.8MiB/s]
100%|██████████| 4.37k/4.37k [00:00<00:00, 17.4kiB/s]
 77%|███████▋  | 280k/361k [00:00<00:00, 2.23MiB/s]
100%|██████████| 361k/361k [00:00<00:00, 973kiB/s] 
100%|██████████| 32.0/32.0 [00:00<00:00, 81.3iB/s]
 82%|████████▏ | 6.37M/7.75M [00:00<00:00, 34.7MiB/s]

 > Model's license - CPML
 > Check https://coqui.ai/cpml.txt for more info.
 > Using model: xtts


100%|██████████| 7.75M/7.75M [00:14<00:00, 34.7MiB/s]/usr/local/lib/python3.11/dist-packages/TTS/tts/layers/xtts/xtts_manager.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental featur

Servidor público do ngrok rodando em: https://0ce7-34-16-195-75.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


...

 > Processing time: 27.218396425247192
 > Real-time factor: 0.6229196501576596
 > Text splitted to sentences.
['Keyditor, você finalmente mudou o seu ponto de renascimento!', 'E o que você fez primeiro?', 'Tomar um banho de sol e comer um pão fresco!', 'É legal que você tenha priorizado essas coisas importantes na vida.', 'E agora que você está no seu novo ponto de renascimento, você está preparado para matar esqueletos e quebrar samambaias.', 'É como se você estivesse em uma verdadeira aventura!']


INFO:werkzeug:127.0.0.1 - - [19/Jan/2025 02:03:09] "POST /synthesize HTTP/1.1" 200 -


 > Processing time: 19.494246006011963
 > Real-time factor: 0.5876352372596184
 > Text splitted to sentences.
['Ahah, Keyditor, você finalmente está fazendo algo útil!', 'Afinal, você não pode ficar apenas matando coisas e comendo pão o tempo todo...', 'Embora, deva admitir, foi divertido assistir àquela vida.', 'Agora, você está criando tábuas de pinheiro, que, você sabe, podem ser usadas para construir coisas, como um novo lar, talvez?', 'Ou, quiçá, uma nova tumba para esconder o corpo que você acabou de...', 'Ah, nunca mind!']


INFO:werkzeug:127.0.0.1 - - [19/Jan/2025 02:04:23] "POST /synthesize HTTP/1.1" 200 -


 > Processing time: 22.95037031173706
 > Real-time factor: 0.5604110543581033


---

Execulte o TTS remoto após terem sido instalados as
dependências